In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import regex as re
from matplotlib import pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', None)

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 16}

plt.rc('font', **font)

### Data Preprocessing/Feature Engineering

In [2]:
#load data
data = pd.read_csv('Merged_Data.csv', index_col = 0)
prov = pd.read_csv('Data/Train-Potential Fraud.csv')
prov.head()

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


In [3]:
data.columns.values

array(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician',
       'DeductibleAmtPaid', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'patientType', 'DOB',
       'Gender', 'Race', 'RenalDiseaseIndicator', 'State', 'County',
       'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
    

In [4]:
prov['PotentialFraud'].value_counts()

No     4904
Yes     506
Name: PotentialFraud, dtype: int64

In [5]:
#convert no to 0 and yes to 1
prov['Fraud'] = np.where(prov['PotentialFraud'] == 'No', 0, 1)
prov = prov.drop('PotentialFraud', axis = 1)

In [6]:
def numeric_col_mean_feature(data_source, by, col, name, target_df):
    '''
    Create a new feature using the mean value grouped on 'by' column from 'data source'
    
    Parameters:
    -----------
    data_source : dataframe used to calculate the mean
    by : string name of column to groupby
    col : string name of column to take mean of
    name : string name of new feature to be created
    target_df : dataframe to join new feature to
    
    Returns:
    -----------
    dataframe containing the new feature joined to the target_df
    '''
    df = data_source.groupby(by)[col].mean()
    df = df.rename(name)
    return target_df.join(df, on = by, how = 'left')

def numeric_col_median_feature(data_source, by, col, name, target_df):
    '''
    Create a new feature using the median value grouped on 'by' column from 'data source'
    
    Parameters:
    -----------
    data_source : dataframe used to calculate the median
    by : string name of column to groupby
    col : string name of column to take median of
    name : string name of new feature to be created
    target_df : dataframe to join new feature to
    
    Returns:
    -----------
    dataframe containing the new feature joined to the target_df
    '''
    df = data_source.groupby(by)[col].median()
    df = df.rename(name)
    return target_df.join(df, on = by, how = 'left')

def numeric_col_max_feature(data_source, by, col, name, target_df):
    '''
    Create a new feature using the max value grouped on 'by' column from 'data source'
    
    Parameters:
    -----------
    data_source : dataframe used to calculate the max
    by : string name of column to groupby
    col : string name of column to take max of
    name : string name of new feature to be created
    target_df : dataframe to join new feature to
    
    Returns:
    -----------
    dataframe containing the new feature joined to the target_df
    '''
    df = data_source.groupby(by)[col].max()
    df = df.rename(name)
    return target_df.join(df, on = by, how = 'left')

Create columns for mean, median, and maximum of the following features in the original data:
- InscClaimAmtReimbursed
- DeductibleAmtPaid
- DaysAdmitted
- NumDiagnoses
- NumProcedures

In [7]:
prov = numeric_col_mean_feature(data, 'Provider', 'InscClaimAmtReimbursed', 'reimburse_mean', prov)
prov = numeric_col_mean_feature(data, 'Provider', 'DeductibleAmtPaid', 'deductible_mean', prov)
prov = numeric_col_mean_feature(data, 'Provider', 'DaysAdmitted', 'days_admitted_mean', prov)
prov = numeric_col_mean_feature(data, 'Provider', 'NumDiagnoses', 'num_diagnoses_mean', prov)
prov = numeric_col_mean_feature(data, 'Provider', 'NumProcedures', 'num_procedures_mean', prov)

In [8]:
prov = numeric_col_median_feature(data, 'Provider', 'InscClaimAmtReimbursed', 'reimburse_median', prov)
prov = numeric_col_median_feature(data, 'Provider', 'DeductibleAmtPaid', 'deductible_median', prov)
prov = numeric_col_median_feature(data, 'Provider', 'DaysAdmitted', 'days_admitted_median', prov)
prov = numeric_col_median_feature(data, 'Provider', 'NumDiagnoses', 'num_diagnoses_median', prov)
prov = numeric_col_median_feature(data, 'Provider', 'NumProcedures', 'num_procedures_median', prov)

In [9]:
prov = numeric_col_max_feature(data, 'Provider', 'InscClaimAmtReimbursed', 'reimburse_max', prov)
prov = numeric_col_max_feature(data, 'Provider', 'DeductibleAmtPaid', 'deductible_max', prov)
prov = numeric_col_max_feature(data, 'Provider', 'DaysAdmitted', 'days_admitted_max', prov)
prov = numeric_col_max_feature(data, 'Provider', 'NumDiagnoses', 'num_diagnoses_max', prov)
prov = numeric_col_max_feature(data, 'Provider', 'NumProcedures', 'num_procedures_max', prov)

In [10]:
prov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Provider               5410 non-null   object 
 1   Fraud                  5410 non-null   int32  
 2   reimburse_mean         5410 non-null   float64
 3   deductible_mean        5409 non-null   float64
 4   days_admitted_mean     5410 non-null   float64
 5   num_diagnoses_mean     5410 non-null   float64
 6   num_procedures_mean    5410 non-null   float64
 7   reimburse_median       5410 non-null   float64
 8   deductible_median      5409 non-null   float64
 9   days_admitted_median   5410 non-null   float64
 10  num_diagnoses_median   5410 non-null   float64
 11  num_procedures_median  5410 non-null   float64
 12  reimburse_max          5410 non-null   int64  
 13  deductible_max         5409 non-null   float64
 14  days_admitted_max      5410 non-null   int64  
 15  num_

In [11]:
#1 deductible is NA, we will fill it with 0
prov = prov.fillna(0)
prov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Provider               5410 non-null   object 
 1   Fraud                  5410 non-null   int32  
 2   reimburse_mean         5410 non-null   float64
 3   deductible_mean        5410 non-null   float64
 4   days_admitted_mean     5410 non-null   float64
 5   num_diagnoses_mean     5410 non-null   float64
 6   num_procedures_mean    5410 non-null   float64
 7   reimburse_median       5410 non-null   float64
 8   deductible_median      5410 non-null   float64
 9   days_admitted_median   5410 non-null   float64
 10  num_diagnoses_median   5410 non-null   float64
 11  num_procedures_median  5410 non-null   float64
 12  reimburse_max          5410 non-null   int64  
 13  deductible_max         5410 non-null   float64
 14  days_admitted_max      5410 non-null   int64  
 15  num_

In [12]:
#rather than DOB, let's create an age column
data_date = dt.datetime(2009,1,1,0,0,0,0)
data['Age'] = round(((data_date - pd.to_datetime(data['DOB'])).dt.days)/365)
prov = numeric_col_mean_feature(data, 'Provider', 'Age', 'age_mean', prov)
prov = numeric_col_median_feature(data, 'Provider', 'Age', 'age_median', prov)

In [13]:
prov

,Provider,Fraud,reimburse_mean,deductible_mean,days_admitted_mean,num_diagnoses_mean,num_procedures_mean,reimburse_median,deductible_median,days_admitted_median,num_diagnoses_median,num_procedures_median,reimburse_max,deductible_max,days_admitted_max,num_diagnoses_max,num_procedures_max,age_mean,age_median
0,PRV51001,0,4185.600000,213.600000,2.440000,3.200000,0.120000,400.0,0.0,1.0,2.0,0.0,42000,1068.0,15,9,2,77.880000,79.0
1,PRV51003,1,4588.409091,502.166667,4.674242,5.250000,0.363636,1750.0,0.0,2.5,4.5,0.0,57000,1068.0,28,10,2,69.083333,71.0
2,PRV51004,0,350.134228,2.080537,2.429530,2.583893,0.000000,70.0,0.0,1.0,2.0,0.0,3300,100.0,21,9,0,71.248322,72.0
3,PRV51005,1,241.124464,3.175966,2.088412,2.588841,0.000000,70.0,0.0,1.0,2.0,0.0,4080,200.0,21,10,0,69.545923,70.0
4,PRV51007,0,468.194444,45.333333,1.958333,2.986111,0.013889,80.0,0.0,1.0,2.0,0.0,10000,1068.0,21,10,1,68.430556,69.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,0,380.000000,4.642857,3.142857,2.071429,0.000000,65.0,0.0,1.0,2.0,0.0,3300,90.0,21,5,0,73.035714,72.5
5406,PRV57760,0,216.818182,0.000000,1.318182,2.363636,0.000000,85.0,0.0,1.0,1.5,0.0,1100,0.0,8,7,0,60.272727,58.0
5407,PRV57761,0,225.243902,4.512195,2.390244,2.670732,0.000000,70.0,0.0,1.0,2.0,0.0,2200,90.0,21,9,0,70.987805,73.0
5408,PRV57762,0,1900.000000,0.000000,1.000000,1.000000,0.000000,1900.0,0.0,1.0,1.0,0.0,1900,0.0,1,1,0,67.000000,67.0


In [14]:
#let's convert patientType column into a numeric column called outpatient where 1 indicates it was an outpatient visit
data.loc[:, 'outpatient'] = np.where(data.loc[:, 'patientType'] == 'outpatient', 0, 1)

#then add proportion of outpatients per provider to provider df
prov = numeric_col_mean_feature(data, 'Provider', 'outpatient', 'outpatient_proportion', prov)
prov.head()

,Provider,Fraud,reimburse_mean,deductible_mean,days_admitted_mean,num_diagnoses_mean,num_procedures_mean,reimburse_median,deductible_median,days_admitted_median,num_diagnoses_median,num_procedures_median,reimburse_max,deductible_max,days_admitted_max,num_diagnoses_max,num_procedures_max,age_mean,age_median,outpatient_proportion
0,PRV51001,0,4185.600000,213.600000,2.440000,3.200000,0.120000,400.0,0.0,1.0,2.0,0.0,42000,1068.0,15,9,2,77.880000,79.0,0.200000
1,PRV51003,1,4588.409091,502.166667,4.674242,5.250000,0.363636,1750.0,0.0,2.5,4.5,0.0,57000,1068.0,28,10,2,69.083333,71.0,0.469697
2,PRV51004,0,350.134228,2.080537,2.429530,2.583893,0.000000,70.0,0.0,1.0,2.0,0.0,3300,100.0,21,9,0,71.248322,72.0,0.000000
3,PRV51005,1,241.124464,3.175966,2.088412,2.588841,0.000000,70.0,0.0,1.0,2.0,0.0,4080,200.0,21,10,0,69.545923,70.0,0.000000
4,PRV51007,0,468.194444,45.333333,1.958333,2.986111,0.013889,80.0,0.0,1.0,2.0,0.0,10000,1068.0,21,10,1,68.430556,69.5,0.041667


In [15]:
#chronic conditions should be 0 = no, 1 = yes, so we'll change 2 to 0
chronic_cols = [col for col in data.columns if 'Chronic' in col]

#let's see if any chronic cols have NA values
data[chronic_cols].isna().sum()

ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
ChronicCond_Depression             0
ChronicCond_Diabetes               0
ChronicCond_IschemicHeart          0
ChronicCond_Osteoporasis           0
ChronicCond_rheumatoidarthritis    0
ChronicCond_stroke                 0
dtype: int64

In [16]:
#since we have no NA values, we can replace all 2's with a 0, and then everything else will be a 1
for i in chronic_cols:
    data.loc[:, i] = np.where(data.loc[:, i] == 2, 0, 1)
data[chronic_cols].head()

,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke
0,1,0,1,0,0,1,1,1,0,1,1
1,1,0,1,0,0,1,1,1,0,1,1
2,1,0,1,0,0,1,1,1,0,1,1
3,0,1,1,0,0,1,1,0,0,1,1
4,0,1,1,0,1,1,0,1,0,0,0


In [17]:
chronic_cols

['ChronicCond_Alzheimer',
 'ChronicCond_Heartfailure',
 'ChronicCond_KidneyDisease',
 'ChronicCond_Cancer',
 'ChronicCond_ObstrPulmonary',
 'ChronicCond_Depression',
 'ChronicCond_Diabetes',
 'ChronicCond_IschemicHeart',
 'ChronicCond_Osteoporasis',
 'ChronicCond_rheumatoidarthritis',
 'ChronicCond_stroke']

In [18]:
#add proportion of patients with each chronic condition for a provider
for i in chronic_cols:
    prov = numeric_col_mean_feature(data, 'Provider', i, str.lower(i), prov)

In [19]:
prov.describe()

,Fraud,reimburse_mean,deductible_mean,days_admitted_mean,num_diagnoses_mean,num_procedures_mean,reimburse_median,deductible_median,days_admitted_median,num_diagnoses_median,num_procedures_median,reimburse_max,deductible_max,days_admitted_max,num_diagnoses_max,num_procedures_max,age_mean,age_median,outpatient_proportion,chroniccond_alzheimer,chroniccond_heartfailure,chroniccond_kidneydisease,chroniccond_cancer,chroniccond_obstrpulmonary,chroniccond_depression,chroniccond_diabetes,chroniccond_ischemicheart,chroniccond_osteoporasis,chroniccond_rheumatoidarthritis,chroniccond_stroke
count,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000
mean,0.093530,1740.679369,155.614405,3.013987,3.407356,0.108011,928.602588,117.811645,1.564510,2.909150,0.062477,13014.913124,446.817745,17.185767,8.150277,0.750277,72.815235,73.702773,0.144568,0.404218,0.594383,0.420224,0.153260,0.322807,0.436831,0.707307,0.765842,0.320718,0.309094,0.104631
std,0.291201,3484.473124,306.468426,2.057721,1.727429,0.246305,2977.988601,332.035542,1.908737,2.102591,0.251225,18995.317219,493.873879,9.227715,2.322526,1.110266,4.712976,4.933886,0.288362,0.182290,0.183746,0.190397,0.133056,0.176796,0.181678,0.168950,0.153917,0.171529,0.168559,0.113676
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,33.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,232.394593,0.311286,1.875000,2.500000,0.000000,70.000000,0.000000,1.000000,2.000000,0.000000,1700.000000,10.000000,10.250000,8.000000,0.000000,70.852941,72.000000,0.000000,0.333333,0.518519,0.333333,0.083333,0.241856,0.363636,0.651475,0.708333,0.250000,0.230769,0.032551
50%,0.000000,356.085106,4.285714,2.586207,2.814550,0.000000,85.000000,0.000000,1.000000,2.000000,0.000000,3300.000000,100.000000,21.000000,9.000000,0.000000,72.939383,74.000000,0.000000,0.400000,0.598485,0.413793,0.142857,0.315165,0.437037,0.714286,0.770492,0.317073,0.307692,0.090909
75%,0.000000,1490.154301,137.363953,3.544613,3.567646,0.083333,100.000000,0.000000,1.000000,3.000000,0.000000,17000.000000,1068.000000,21.000000,10.000000,2.000000,74.849389,76.000000,0.128205,0.484848,0.666667,0.500000,0.197452,0.392494,0.500000,0.785714,0.839304,0.385388,0.375000,0.137931
max,1.000000,57000.000000,1068.000000,36.000000,10.000000,3.000000,57000.000000,1068.000000,36.000000,10.000000,3.000000,125000.000000,1068.000000,37.000000,10.000000,5.000000,100.000000,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
#we should also include proportion of patients with renal disease that a provider has seen
data['RenalDiseaseIndicator'].value_counts()

0    448363
Y    109848
Name: RenalDiseaseIndicator, dtype: int64

In [21]:
#convert 'Y' to 1
data.loc[:, 'RenalDiseaseIndicator'] = np.where(data['RenalDiseaseIndicator'] == 'Y', 1, 0)
prov = numeric_col_mean_feature(data, 'Provider', 'RenalDiseaseIndicator', 'renal_disease', prov)

In [22]:
prov.describe()

,Fraud,reimburse_mean,deductible_mean,days_admitted_mean,num_diagnoses_mean,num_procedures_mean,reimburse_median,deductible_median,days_admitted_median,num_diagnoses_median,num_procedures_median,reimburse_max,deductible_max,days_admitted_max,num_diagnoses_max,num_procedures_max,age_mean,age_median,outpatient_proportion,chroniccond_alzheimer,chroniccond_heartfailure,chroniccond_kidneydisease,chroniccond_cancer,chroniccond_obstrpulmonary,chroniccond_depression,chroniccond_diabetes,chroniccond_ischemicheart,chroniccond_osteoporasis,chroniccond_rheumatoidarthritis,chroniccond_stroke,renal_disease
count,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000,5410.000000
mean,0.093530,1740.679369,155.614405,3.013987,3.407356,0.108011,928.602588,117.811645,1.564510,2.909150,0.062477,13014.913124,446.817745,17.185767,8.150277,0.750277,72.815235,73.702773,0.144568,0.404218,0.594383,0.420224,0.153260,0.322807,0.436831,0.707307,0.765842,0.320718,0.309094,0.104631,0.196768
std,0.291201,3484.473124,306.468426,2.057721,1.727429,0.246305,2977.988601,332.035542,1.908737,2.102591,0.251225,18995.317219,493.873879,9.227715,2.322526,1.110266,4.712976,4.933886,0.288362,0.182290,0.183746,0.190397,0.133056,0.176796,0.181678,0.168950,0.153917,0.171529,0.168559,0.113676,0.147006
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,33.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,232.394593,0.311286,1.875000,2.500000,0.000000,70.000000,0.000000,1.000000,2.000000,0.000000,1700.000000,10.000000,10.250000,8.000000,0.000000,70.852941,72.000000,0.000000,0.333333,0.518519,0.333333,0.083333,0.241856,0.363636,0.651475,0.708333,0.250000,0.230769,0.032551,0.125000
50%,0.000000,356.085106,4.285714,2.586207,2.814550,0.000000,85.000000,0.000000,1.000000,2.000000,0.000000,3300.000000,100.000000,21.000000,9.000000,0.000000,72.939383,74.000000,0.000000,0.400000,0.598485,0.413793,0.142857,0.315165,0.437037,0.714286,0.770492,0.317073,0.307692,0.090909,0.188265
75%,0.000000,1490.154301,137.363953,3.544613,3.567646,0.083333,100.000000,0.000000,1.000000,3.000000,0.000000,17000.000000,1068.000000,21.000000,10.000000,2.000000,74.849389,76.000000,0.128205,0.484848,0.666667,0.500000,0.197452,0.392494,0.500000,0.785714,0.839304,0.385388,0.375000,0.137931,0.250000
max,1.000000,57000.000000,1068.000000,36.000000,10.000000,3.000000,57000.000000,1068.000000,36.000000,10.000000,3.000000,125000.000000,1068.000000,37.000000,10.000000,5.000000,100.000000,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
data['Gender'].value_counts()

2    323114
1    235097
Name: Gender, dtype: int64

In [24]:
#let's change gender 2 to gender 0 and then convert gender into proportion of patients seen of gender 1
data.loc[:, 'Gender'] = np.where(data.loc[:, 'Gender'] == 2, 0, 1)
prov = numeric_col_mean_feature(data, 'Provider', 'Gender', 'gender', prov)
prov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5410 entries, 0 to 5409
Data columns (total 33 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Provider                         5410 non-null   object 
 1   Fraud                            5410 non-null   int32  
 2   reimburse_mean                   5410 non-null   float64
 3   deductible_mean                  5410 non-null   float64
 4   days_admitted_mean               5410 non-null   float64
 5   num_diagnoses_mean               5410 non-null   float64
 6   num_procedures_mean              5410 non-null   float64
 7   reimburse_median                 5410 non-null   float64
 8   deductible_median                5410 non-null   float64
 9   days_admitted_median             5410 non-null   float64
 10  num_diagnoses_median             5410 non-null   float64
 11  num_procedures_median            5410 non-null   float64
 12  reimburse_max       

In [25]:
data.Race.value_counts()

1    471036
2     55640
3     19715
5     11820
Name: Race, dtype: int64

In [26]:
#rename category 5 as 4
data.loc[:, 'Race'] = np.where(data.loc[:, 'Race'] == 5, 4, data.loc[:, 'Race'])

In [27]:
data.Race.value_counts()

1    471036
2     55640
3     19715
4     11820
Name: Race, dtype: int64

In [28]:
#initialize race columns as 0
prov['race_1'] = 0
prov['race_2'] = 0
prov['race_3'] = 0
prov['race_4'] = 0

#calculate proportion of patients by race for each provider
race_proportions = data.groupby('Provider')['Race'].value_counts() / data.groupby('Provider')['Race'].count()

#set index to provider so df is easier to update
prov = prov.set_index('Provider')

#fill the race columns by provider and race
for i in race_proportions.index:
    col = 'race_' + str(i[1])
    prov.loc[i[0], col] = race_proportions[i]

In [29]:
#check to make sure each provider's race proportions sum up to 1, and with 5410 providers this should be 5410
sum(prov[['race_1', 'race_2', 'race_3', 'race_4']].sum(axis = 1))

5410.0

In [30]:
prov.head()

,Fraud,reimburse_mean,deductible_mean,days_admitted_mean,num_diagnoses_mean,num_procedures_mean,reimburse_median,deductible_median,days_admitted_median,num_diagnoses_median,num_procedures_median,reimburse_max,deductible_max,days_admitted_max,num_diagnoses_max,num_procedures_max,age_mean,age_median,outpatient_proportion,chroniccond_alzheimer,chroniccond_heartfailure,chroniccond_kidneydisease,chroniccond_cancer,chroniccond_obstrpulmonary,chroniccond_depression,chroniccond_diabetes,chroniccond_ischemicheart,chroniccond_osteoporasis,chroniccond_rheumatoidarthritis,chroniccond_stroke,renal_disease,gender,race_1,race_2,race_3,race_4
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV51001,0,4185.600000,213.600000,2.440000,3.200000,0.120000,400.0,0.0,1.0,2.0,0.0,42000,1068.0,15,9,2,77.880000,79.0,0.200000,0.600000,0.760000,0.680000,0.200000,0.400000,0.360000,0.840000,0.920000,0.240000,0.320000,0.240000,0.320000,0.360000,0.840000,0.160000,0.000000,0.000000
PRV51003,1,4588.409091,502.166667,4.674242,5.250000,0.363636,1750.0,0.0,2.5,4.5,0.0,57000,1068.0,28,10,2,69.083333,71.0,0.469697,0.424242,0.606061,0.484848,0.075758,0.310606,0.409091,0.757576,0.848485,0.250000,0.287879,0.090909,0.219697,0.409091,0.810606,0.181818,0.000000,0.007576
PRV51004,0,350.134228,2.080537,2.429530,2.583893,0.000000,70.0,0.0,1.0,2.0,0.0,3300,100.0,21,9,0,71.248322,72.0,0.000000,0.429530,0.590604,0.335570,0.107383,0.275168,0.422819,0.704698,0.724832,0.328859,0.308725,0.114094,0.154362,0.308725,0.805369,0.161074,0.033557,0.000000
PRV51005,1,241.124464,3.175966,2.088412,2.588841,0.000000,70.0,0.0,1.0,2.0,0.0,4080,200.0,21,10,0,69.545923,70.0,0.000000,0.365665,0.583691,0.435193,0.141631,0.253219,0.416309,0.685837,0.768240,0.295279,0.284120,0.106438,0.222318,0.438627,0.766524,0.224893,0.008584,0.000000
PRV51007,0,468.194444,45.333333,1.958333,2.986111,0.013889,80.0,0.0,1.0,2.0,0.0,10000,1068.0,21,10,1,68.430556,69.5,0.041667,0.361111,0.555556,0.305556,0.166667,0.222222,0.402778,0.680556,0.708333,0.291667,0.305556,0.166667,0.152778,0.472222,0.805556,0.194444,0.000000,0.000000


In [31]:
#add total number of counties a provider services
num_counties = data.groupby(['Provider', 'County'])['County'].count().reset_index(name='a'). \
groupby('Provider')['County'].count().reset_index(name='num_counties')

In [32]:
prov = pd.merge(prov, num_counties, on = 'Provider')

In [33]:
#add total number of states a provider services
num_states = data.groupby(['Provider', 'State'])['State'].count().reset_index(name='b'). \
groupby('Provider')['State'].count().reset_index(name = 'num_states')

In [34]:
prov = pd.merge(prov, num_states, on = 'Provider')

In [35]:
prov.groupby('Fraud')['num_counties'].mean()

Fraud
0     7.537520
1    21.249012
Name: num_counties, dtype: float64

In [36]:
prov.groupby('Fraud')['num_states'].mean()

Fraud
0    2.283646
1    5.379447
Name: num_states, dtype: float64

We can see that fraudulent providers tend to provide in higher numbers of states and counties than non-fraudulent providers.

In [37]:
#count number of chronic conditions patients have
num_chronic = data.filter(regex='Chronic').sum(axis=1).reset_index(name='num_chronic_cond')
num_chronic = num_chronic.drop('index', axis = 1)
num_chronic = pd.concat([num_chronic, data[['Provider', 'ClaimID']]], axis=1)

In [38]:
#use number of chronic conditions to count average number of conditions that a provider's patients have
avg_chronic = num_chronic.groupby(['Provider', 'num_chronic_cond'])['ClaimID'].count().reset_index(name='num_claims').\
groupby('Provider').agg({'num_chronic_cond':'mean', 'num_claims':'mean'}).reset_index().\
rename(columns={'num_chronic_cond':'avg_chronic', 'num_claims':'avg_claim'})

In [39]:
prov = pd.merge(prov, avg_chronic, on = 'Provider')

In [40]:
prov.groupby('Fraud')['avg_claim'].mean()

Fraud
0     7.213598
1    37.868535
Name: avg_claim, dtype: float64

In [41]:
prov.groupby('Fraud')['avg_chronic'].mean()

Fraud
0    4.584667
1    4.972209
Name: avg_chronic, dtype: float64

Chronic conditions don't seem to indicate fraud, but number of claims seems to be a strong indicator.

In [42]:
#let's look at how many times providers file claims that have the exact same dx and px codes
#df with all duplicated claims:
duplicates = data[data.duplicated(['ClmDiagnosisCode_1', \
                      'ClmDiagnosisCode_2', \
                      'ClmDiagnosisCode_3', \
                      'ClmDiagnosisCode_4', \
                      'ClmDiagnosisCode_5', \
                      'ClmDiagnosisCode_6', \
                      'ClmDiagnosisCode_7', \
                      'ClmDiagnosisCode_8', \
                      'ClmDiagnosisCode_9', \
                      'ClmDiagnosisCode_10', \
                      'ClmProcedureCode_1', \
                      'ClmProcedureCode_2', \
                      'ClmProcedureCode_3', \
                      'ClmProcedureCode_4', \
                      'ClmProcedureCode_5', \
                      'ClmProcedureCode_6'], \
                     keep = False)]

In [43]:
#find claims that have more than one of the same type of claim
duplicate_count = duplicates.groupby('Provider')['BeneID'].value_counts()
duplicate_count = duplicate_count[duplicate_count > 1].groupby('Provider').count().reset_index(name = "duplicated_claims")

In [44]:
prov = pd.merge(prov, duplicate_count, on = 'Provider', how = 'left')

In [45]:
#fill in empty values with 0 as that means they don't have any duplicates
prov['duplicated_claims'].fillna(0, inplace = True)

In [46]:
prov.groupby('Fraud')['duplicated_claims'].mean()

Fraud
0     4.294046
1    32.944664
Name: duplicated_claims, dtype: float64

Duplicates seem to be a good way to identify fraud

In [47]:
#check which providers have the same attending physician repeated
duplicate_attending = duplicates.groupby('Provider')['AttendingPhysician'].value_counts()
duplicate_attending = duplicate_attending[duplicate_attending > 1]
duplicate_attending = duplicate_attending.groupby('Provider').count().reset_index(name = 'duplicate_attending')

In [48]:
prov = pd.merge(prov, duplicate_attending, on = 'Provider', how = 'left')

In [49]:
prov = prov.fillna(0)

In [50]:
prov.groupby('Fraud')['duplicate_attending'].mean()

Fraud
0     3.920065
1    13.027668
Name: duplicate_attending, dtype: float64

In [51]:
cor = prov.corr()

In [52]:
#let's select features that are most impactful to our target, Fraud
high_cor = cor[abs(cor['Fraud']) > 0.2][['Fraud']]
high_cor

,Fraud
Fraud,1.000000
deductible_mean,0.204640
reimburse_max,0.514671
deductible_max,0.326441
days_admitted_max,0.375310
num_diagnoses_max,0.235075
num_procedures_max,0.453090
outpatient_proportion,0.205683
num_counties,0.372852
num_states,0.295204


In [53]:
train = prov[high_cor.index.values]

In [54]:
#deductible has both mean and max which are highly correlated to each other, we'll drop the mean
train = train.drop('deductible_mean', axis = 1)

In [55]:
train.shape

(5410, 12)

Let's train a preliminary model to see how it fits using the features we have engineered

In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X = train.iloc[:, 1:]
y = train.iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred = rfc.predict_proba(X_test)[:, 1]

In [57]:
from sklearn.metrics import confusion_matrix, classification_report
#print classification report and confusion matrix for various probability thresholds to see how model fits
for i in np.linspace(0.05, 0.95, 19):
    print('Threshold', round(i, 3))
    yp = np.where(y_pred >= round(i, 3), 1, 0)
    print(classification_report(y_test, yp))
    print(confusion_matrix(y_test, yp))
    print('#######################################################')

Threshold 0.05
              precision    recall  f1-score   support

           0       0.99      0.81      0.89       979
           1       0.34      0.94      0.50       103

    accuracy                           0.82      1082
   macro avg       0.67      0.87      0.69      1082
weighted avg       0.93      0.82      0.85      1082

[[790 189]
 [  6  97]]
#######################################################
Threshold 0.1
              precision    recall  f1-score   support

           0       0.99      0.88      0.93       979
           1       0.45      0.93      0.61       103

    accuracy                           0.88      1082
   macro avg       0.72      0.91      0.77      1082
weighted avg       0.94      0.88      0.90      1082

[[861 118]
 [  7  96]]
#######################################################
Threshold 0.15
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       979
           1       0.50      0.90  

In [59]:
#check feature importance
test = pd.Series(sorted(list(zip(X_train.columns, rfc.feature_importances_)), key=lambda x:x[1], reverse = True))
test

0         (days_admitted_max, 0.21702064828897524)
1             (reimburse_max, 0.17189460288465902)
2                   (avg_claim, 0.144876103462972)
3         (duplicated_claims, 0.08848327046558109)
4     (outpatient_proportion, 0.08272296943692528)
5              (num_counties, 0.07744370440836694)
6        (num_procedures_max, 0.06003832920269856)
7       (duplicate_attending, 0.05788048449857425)
8                 (num_states, 0.0483702513917051)
9         (num_diagnoses_max, 0.03498579547854861)
10           (deductible_max, 0.01628384048099383)
dtype: object

In [62]:
prov.to_csv('provider_features.csv')
train.to_csv('train_data.csv')